# 스타벅스 매장정보 수집

In [ ]:
! pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 25.3 MB/s 
     |████████████████████████████████| 358 kB 41.4 MB/s 
     |████████████████████████████████| 139 kB 41.8 MB/s 
     |████████████████████████████████| 55 kB 5.1 MB/s 
     |████████████████████████████████| 4.1 MB 55.1 MB/s 
     |████████████████████████████████| 58 kB 5.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install webdriver_manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 49.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.10
    Uninstalling urllib3-1.26.10:
      Successfully uninstalled urllib3-1.26.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.3.0 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://www.istarbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)
time.sleep(3)
driver.find_element(by=By.XPATH, value="//*[@id='container']/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[4]/a").click()
time.sleep(1)
driver.find_element(by=By.XPATH, value="//*[@id='mCSB_2_container']/ul/li[1]/a").click()
time.sleep(2)
starbucks_list = driver.find_elements(by=By.CLASS_NAME, value="quickResultLstCon")
print(len(starbucks_list))
starbucks = []

for i, store in enumerate(starbucks_list):
    name = store.get_attribute("data-name")
    lat = store.get_attribute("data-lat")
    lng = store.get_attribute("data-long")
    address = driver.find_element(by=By.XPATH, value=f"//*[@id='mCSB_3_container']/ul/li[{i+1}]/p").get_attribute('innerText').split('\n')[0]
    starbucks.append([name, lat, lng, address])

C:\Users\ooroo\AppData\Local\Temp/ipykernel_15264/2457302748.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


76


In [ ]:
columns = ['매장명', '위도', '경도', '주소']
starbucks_df = pd.DataFrame(starbucks, columns=columns)
starbucks_df.head()

,매장명,위도,경도,주소
0,대구영대병원역DT,35.845522,128.593536,대구광역시 남구 봉덕로 25 (봉덕동)
1,대구앞산DT,35.835516,128.579871,대구광역시 남구 현충로 32 (대명동)
2,대구가톨릭대학교병원,35.84339006877753,128.56742153199514,대구광역시 남구 두류공원로17길 33 (대명동)
3,계명대동산병원,35.853811,128.480276,대구광역시 달서구 달구벌대로 1035 (신당동)
4,대구죽전역,35.848437,128.536007,대구광역시 달서구 와룡로 169


In [ ]:
starbucks_df.to_excel('./starbucks.xlsx', index=False)

# 대구시 인구 데이터

In [ ]:
daegu_pop = pd.read_csv('./daegupop.csv', encoding="cp949", header=1)

In [ ]:
daegu_pop.head()

,행정구역(구·군)별(1),5세별(1),계,남자,여자,계.1,남자.1,여자.1
0,합계,계,27798,14911,12887,27254,14405,12849
1,합계,0 - 4세,572,266,306,504,240,264
2,합계,5 - 9세,381,164,217,433,200,233
3,합계,10 - 14세,289,145,144,301,144,157
4,합계,15 - 19세,1341,642,699,390,182,208


In [ ]:
daegu_pop = daegu_pop.loc[:, :'여자']

In [ ]:
daegu_pop.columns=['행정구역', '연령', '총인구', '남자', '여자']

In [ ]:
daegu_pop = daegu_pop[daegu_pop['행정구역']!='합계']

In [ ]:
daegu_pop['행정구역'].unique()

array(['중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군'], dtype=object)

In [ ]:
daegu_pop.reset_index(inplace=True, drop=True)

In [ ]:
daegu_pop.to_excel('./daegu_pop.xlsx', index=False)

# 대구시 사업체 데이터

In [ ]:
stores = pd.read_csv('stores.csv', encoding='cp949')

In [ ]:
stores = stores.transpose()

In [ ]:
stores = stores.loc[['행정구역(구군)별(1)', '2019', '2019.2'], :]

In [ ]:
stores

,0,1,2,3,4,5,6,7,8,9,10
행정구역(구군)별(1),행정구역(구군)별(1),행정구역(구군)별(1),합계,중구,동구,서구,남구,북구,수성구,달서구,달성군
2019,합계,사업체수 (개),210944,21411,26377,18699,14141,41364,27563,42679,18710
2019.2,합계,종사자수 (명),967934,90671,117723,78524,57702,169609,133665,213918,106122


In [ ]:
stores.set_index(keys=[1], inplace=True, drop=True)

In [ ]:
stores

,0,2,3,4,5,6,7,8,9,10
1,,,,,,,,,,
행정구역(구군)별(1),행정구역(구군)별(1),합계,중구,동구,서구,남구,북구,수성구,달서구,달성군
사업체수 (개),합계,210944,21411,26377,18699,14141,41364,27563,42679,18710
종사자수 (명),합계,967934,90671,117723,78524,57702,169609,133665,213918,106122


In [ ]:
stores.reset_index(drop=False, inplace=True)

In [ ]:
stores

,1,0,2,3,4,5,6,7,8,9,10
0,행정구역(구군)별(1),행정구역(구군)별(1),합계,중구,동구,서구,남구,북구,수성구,달서구,달성군
1,사업체수 (개),합계,210944,21411,26377,18699,14141,41364,27563,42679,18710
2,종사자수 (명),합계,967934,90671,117723,78524,57702,169609,133665,213918,106122


In [ ]:
stores.columns = stores.iloc[0]

In [ ]:
stores

,행정구역(구군)별(1),행정구역(구군)별(1),합계,중구,동구,서구,남구,북구,수성구,달서구,달성군
0,행정구역(구군)별(1),행정구역(구군)별(1),합계,중구,동구,서구,남구,북구,수성구,달서구,달성군
1,사업체수 (개),합계,210944,21411,26377,18699,14141,41364,27563,42679,18710
2,종사자수 (명),합계,967934,90671,117723,78524,57702,169609,133665,213918,106122


In [ ]:
stores = stores.iloc[1:]

In [ ]:
stores = stores.transpose()

In [ ]:
stores

,1,2
0,,
행정구역(구군)별(1),사업체수 (개),종사자수 (명)
행정구역(구군)별(1),합계,합계
합계,210944,967934
중구,21411,90671
동구,26377,117723
서구,18699,78524
남구,14141,57702
북구,41364,169609
수성구,27563,133665


In [ ]:
stores.reset_index(drop=False, inplace=True)

In [ ]:
stores.drop(stores.index[1], inplace=True)

In [ ]:
stores.columns = stores.iloc[0]

In [ ]:
stores.drop(stores.index[0], inplace=True)

In [ ]:
stores.columns=['행정구역', '사업체수', '종사자수']

In [ ]:
stores.reset_index(drop=True, inplace=True)

In [ ]:
stores

,행정구역,사업체수,종사자수
0,합계,210944,967934
1,중구,21411,90671
2,동구,26377,117723
3,서구,18699,78524
4,남구,14141,57702
5,북구,41364,169609
6,수성구,27563,133665
7,달서구,42679,213918
8,달성군,18710,106122


In [ ]:
stores.to_excel('./daegu_stores.xlsx', index=False)

# 대구시 행정구역 데이터

In [ ]:
daegu = pd.read_excel('sgg.xlsx')

In [ ]:
daegu.head()

,docity,do,city,longitude,latitude
0,강원강릉시,강원,강릉시,128.878497,37.749136
1,강원고성군,강원,고성군,128.470164,38.377961
2,강원동해시,강원,동해시,129.116633,37.521931
3,강원삼척시,강원,삼척시,129.167489,37.447086
4,강원속초시,강원,속초시,128.594167,38.204275


In [ ]:
daegu = daegu[daegu['do']=='대구시']

In [ ]:
daegu.drop(daegu.index[0], inplace=True)

In [ ]:
daegu.reset_index(drop=True, inplace=True)

In [ ]:
daegu

,docity,do,city,longitude,latitude
0,대구시수성구,대구시,수성구,128.632867,35.855208
1,대구시남구,대구시,남구,128.597702,35.846214
2,대구시달서구,대구시,달서구,128.532590,35.829977
3,대구시달성군,대구시,달성군,128.431399,35.774750
4,대구시동구,대구시,동구,128.635558,35.886827
5,대구시북구,대구시,북구,128.582892,35.885865
6,대구시서구,대구시,서구,128.559160,35.871941
7,대구시수성구,대구시,수성구,128.630701,35.858351
8,대구시중구,대구시,중구,128.606174,35.869527


In [ ]:
daegu.to_excel('./daegu.xlsx', index=False)

# 데이터 합치기

In [ ]:
starbucks = pd.read_excel('starbucks.xlsx')
stores = pd.read_excel('daegu_stores.xlsx')
daegu =  pd.read_excel('daegu.xlsx')
pop =  pd.read_excel('daegu_pop.xlsx')

In [ ]:
starbucks.head()

,매장명,위도,경도,주소
0,대구영대병원역DT,35.845522,128.593536,대구광역시 남구 봉덕로 25 (봉덕동)
1,대구앞산DT,35.835516,128.579871,대구광역시 남구 현충로 32 (대명동)
2,대구가톨릭대학교병원,35.843390,128.567422,대구광역시 남구 두류공원로17길 33 (대명동)
3,계명대동산병원,35.853811,128.480276,대구광역시 달서구 달구벌대로 1035 (신당동)
4,대구죽전역,35.848437,128.536007,대구광역시 달서구 와룡로 169


In [ ]:
sggs = []
for address in starbucks['주소']:
    sgg = address.split()[1]
    sggs.append(sgg)
starbucks['행정구역'] = sggs
starbucks.head()

,매장명,위도,경도,주소,행정구역
0,대구영대병원역DT,35.845522,128.593536,대구광역시 남구 봉덕로 25 (봉덕동),남구
1,대구앞산DT,35.835516,128.579871,대구광역시 남구 현충로 32 (대명동),남구
2,대구가톨릭대학교병원,35.843390,128.567422,대구광역시 남구 두류공원로17길 33 (대명동),남구
3,계명대동산병원,35.853811,128.480276,대구광역시 달서구 달구벌대로 1035 (신당동),달서구
4,대구죽전역,35.848437,128.536007,대구광역시 달서구 와룡로 169,달서구


In [ ]:
starbucks_sgg_count = starbucks.pivot_table(index='행정구역', 
                                           values = '매장명',
                                           aggfunc = 'count')

In [ ]:
starbucks_sgg_count.rename(columns = {'매장명':'매장수'}, inplace=True)

In [ ]:
starbucks_sgg_count.reset_index(drop=False, inplace=True)

In [ ]:
starbucks_sgg_count = starbucks_sgg_count[1:]

In [ ]:
starbucks_sgg_count

,행정구역,매장수
1,남구,3
2,달서구,18
3,달성군,2
4,동구,11
5,북구,9
6,서구,1
7,수성구,21
8,중구,10


In [ ]:
daegu.rename(columns={'city':'행정구역'}, inplace=True)

In [ ]:
daegu

,docity,do,행정구역,longitude,latitude
0,대구시수성구,대구시,수성구,128.632867,35.855208
1,대구시남구,대구시,남구,128.597702,35.846214
2,대구시달서구,대구시,달서구,128.532590,35.829977
3,대구시달성군,대구시,달성군,128.431399,35.774750
4,대구시동구,대구시,동구,128.635558,35.886827
5,대구시북구,대구시,북구,128.582892,35.885865
6,대구시서구,대구시,서구,128.559160,35.871941
7,대구시수성구,대구시,수성구,128.630701,35.858351
8,대구시중구,대구시,중구,128.606174,35.869527


In [ ]:
daegu = pd.merge(daegu, starbucks_sgg_count, how='left', on='행정구역')

In [ ]:
daegu

,docity,do,행정구역,longitude,latitude,매장수
0,대구시수성구,대구시,수성구,128.632867,35.855208,21
1,대구시남구,대구시,남구,128.597702,35.846214,3
2,대구시달서구,대구시,달서구,128.532590,35.829977,18
3,대구시달성군,대구시,달성군,128.431399,35.774750,2
4,대구시동구,대구시,동구,128.635558,35.886827,11
5,대구시북구,대구시,북구,128.582892,35.885865,9
6,대구시서구,대구시,서구,128.559160,35.871941,1
7,대구시수성구,대구시,수성구,128.630701,35.858351,21
8,대구시중구,대구시,중구,128.606174,35.869527,10


In [ ]:
daegu = pd.merge(daegu, pop, how='left', on='행정구역')

In [ ]:
daegu = pd.merge(daegu, stores, how='left', on='행정구역')

In [ ]:
daegu.to_excel('daegu_starbucks.xlsx', index=False)

# 시각화

In [ ]:
import folium
import json
import os

In [ ]:
starbucks_map = folium.Map(location = [35.8714, 128.6014],
                          tiles = 'Stamen Terrain',
                          zoom_start = 11)

In [ ]:
starbucks_map

In [ ]:
for idx in starbucks.index:
    lat = starbucks.loc[idx, '위도']
    lng = starbucks.loc[idx, '경도']
    
    folium.CircleMarker(location=[lat, lng],
                       fill = True,
                       fill_color = 'green',
                       fill_opacity = 1,
                       color = 'yellow', 
                       weight = 1,
                       radius = 3).add_to(starbucks_map)

In [ ]:
starbucks_map

# 행정구역별 매장수

In [ ]:
!git clone https://github.com/raqoon886/Local_HangJeongDong.git
os.chdir('./Local_HangJeongDong')

with open('./hangjeongdong_대구광역시.geojson', 'r') as f:
    daegu_geo = json.load(f)

Cloning into 'Local_HangJeongDong'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 59 (delta 28), reused 44 (delta 16), pack-reused 0
Unpacking objects: 100% (59/59), done.


In [ ]:
starbucks_bubble = folium.Map(location = [35.8714, 128.6014],
                          tiles = 'CartoDB dark_matter',
                          zoom_start = 11)

In [ ]:
def style_function(feature):
    return {
        'opacity' : 0.7, 
        'weight' : 1, 
        'color' : 'white', 
        'fillOpacity' : 0,
        'dashArray' : '5, 5',
    }

folium.GeoJson(
    daegu_geo,
    style_function = style_function).add_to(starbucks_bubble)

starbucks_bubble

In [ ]:
daegu_1 = daegu[daegu['연령']=='계']

In [ ]:
daegu_1 = daegu_1.astype({'longitude':int,	'latitude':int,	'매장수':int,	'총인구':int,	'남자':int,	'여자':int,	'사업체수':int,	'종사자수':int})

In [ ]:
daegu_1

,docity,do,행정구역,longitude,latitude,매장수,연령,총인구,남자,여자,사업체수,종사자수
0,대구시수성구,대구시,수성구,128,35,21,계,1673,690,983,27563,133665
23,대구시남구,대구시,남구,128,35,3,계,1615,699,916,14141,57702
46,대구시달서구,대구시,달서구,128,35,18,계,9850,5764,4086,42679,213918
69,대구시달성군,대구시,달성군,128,35,2,계,5341,3417,1924,18710,106122
92,대구시동구,대구시,동구,128,35,11,계,1765,591,1174,26377,117723
115,대구시북구,대구시,북구,128,35,9,계,4660,2219,2441,41364,169609
138,대구시서구,대구시,서구,128,35,1,계,2179,1198,981,18699,78524
161,대구시수성구,대구시,수성구,128,35,21,계,1673,690,983,27563,133665
184,대구시중구,대구시,중구,128,35,10,계,715,333,382,21411,90671


In [ ]:
starbucks_mean = daegu_1['매장수'].mean()
starbucks_mean

10.666666666666666

In [ ]:
for idx in daegu_1.index:
    lat = daegu_1.loc[idx, 'latitude']
    lng = daegu_1.loc[idx, 'longitude']
    count = daegu_1.loc[idx, '매장수']
    
    if count>starbucks_mean:
        fillColor = 'red'
    else:
        fillColor = 'blue'
        
    folium.CircleMarker(location = [lat, lng],
                       color = '#FFFF00',
                       fill_color = fillColor,
                       fill_opacity = 0.7,
                       weight = 1.5,
                       radius = count/1).add_to(starbucks_bubble)
    
starbucks_bubble
    

# 행정구역별 매장수와 인구수 비교

In [ ]:
starbucks_choropleth_1 = folium.Map(location = [35.8714, 128.6014],
                          tiles = 'CartoDB dark_matter',
                          zoom_start = 11)

folium.Choropleth(geo_data = daegu_geo,
                 data = daegu_1,
                 columns = ['행정구역', '매장수'], # 첫번째값은 지도 데이터와 매칭 (key_on)
                 fill_color = 'YlGn',
                 fill_opacity = 0.7, 
                 line_opacity = 0.5,
                 key_on = 'properties.sggnm').add_to(starbucks_choropleth_1)
starbucks_choropleth_1.save(outfile='sggstarbucks.html')
starbucks_choropleth_1


In [ ]:
starbucks_choropleth_2 = folium.Map(location = [35.8714, 128.6014],
                          tiles = 'CartoDB dark_matter',
                          zoom_start = 11)

folium.Choropleth(geo_data = daegu_geo,
                 data = daegu_1,
                 columns = ['행정구역', '총인구'], # 첫번째값은 지도 데이터와 매칭 (key_on)
                 fill_color = 'YlGn',
                 fill_opacity = 0.7, 
                 line_opacity = 0.5,
                 key_on = 'properties.sggnm').add_to(starbucks_choropleth_2)
starbucks_choropleth_2.save(outfile='population.html')
starbucks_choropleth_2

# 행정구역별 매장수와 사업체수 비교

In [ ]:
starbucks_choropleth_3 = folium.Map(location = [35.8714, 128.6014],
                          tiles = 'CartoDB dark_matter',
                          zoom_start = 11)

folium.Choropleth(geo_data = daegu_geo,
                 data = daegu_1,
                 columns = ['행정구역', '사업체수'], # 첫번째값은 지도 데이터와 매칭 (key_on)
                 fill_color = 'YlGn',
                 fill_opacity = 0.7, 
                 line_opacity = 0.5,
                 key_on = 'properties.sggnm').add_to(starbucks_choropleth_3)
starbucks_choropleth_3.save('sggstores.html')
starbucks_choropleth_3